In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import itertools

In [2]:
from bs4 import BeautifulSoup as BS
import requests

In [3]:
import warnings
warnings.simplefilter(action="ignore")

In [4]:
companies = ["AMAZON", "APPLE", "GOOGLE", "META", "NETFLIX"]
    
URLS = ['https://finance.yahoo.com/quote/AMZN', 'https://finance.yahoo.com/quote/AAPL',
        'https://finance.yahoo.com/quote/GOOG', 'https://finance.yahoo.com/quote/META', 
        'https://finance.yahoo.com/quote/NFLX']

time_period = ["daily"]

In [5]:
for n, (company, period) in enumerate(list(itertools.product(companies, time_period))):
    
    date_parse = lambda dates: pd.to_datetime(dates)
    path = f"/diploma_info/datalake/raw_data/{company}_{period}.csv"

    full_set = pd.read_csv(
        path,
        parse_dates=["Date"],
        date_parser=date_parse,
        index_col=["Date"],
    )
    
    date_1 = full_set.index[-1]
    if (date_1+timedelta(days=2)).weekday() == 5 or \
       (date_1+timedelta(days=2)).weekday() == 6:
        date_2 = date_1 + timedelta(days=4)
    else:
        date_2 = date_1 + timedelta(days=2)
        
    new_dates = pd.date_range(date_1, date_2, freq='B')[1:]
    full_set = pd.concat([full_set, pd.DataFrame(None, index=new_dates)])
    
    full_set.index.name = 'date'
    full_set.columns = ["_".join(c.lower().split()) for c in full_set.columns]

    req_1 = requests.get(URLS[n])
    soup = BS(req_1.text, "html.parser")
    full_set.loc[date_2, 'open'] = float(soup.find_all('fin-streamer', class_="svelte-tx3nkj")[1].text.strip())
    full_set.loc[new_dates[0], 'open'] = full_set.shift(1).loc[new_dates[0], 'close']

        
    full_set['diff_close_value'] = full_set['close'].diff()
    full_set['diff_open_value'] = full_set['open'].diff()
    full_set['open-prev_close'] = full_set['open'] - full_set['close'].shift(1)
    full_set['(close-open)_lag_1'] = (full_set['close'] - full_set['open']).shift(1)
    
    full_set.loc[full_set[full_set['diff_close_value'] > 0].index, 'growth'] = 1
    full_set.loc[full_set[full_set['diff_open_value'] > 0].index, 'growth_open'] = 1
    full_set['growth_lag_1'] = full_set['growth'].shift(1).bfill()
    
    full_set[f'diff_open_value_mean_3_days'] = abs(full_set['diff_open_value'].rolling(window=3, closed='left').agg('mean'))
    full_set.loc[full_set['growth_open'] == 0, f'diff_open_value_mean_3_days'] = \
    -(full_set.loc[full_set['growth_open'] == 0, f'diff_open_value_mean_3_days'].values)
    full_set['new_open'] = full_set['open'].shift(1) + full_set['diff_open_value_mean_3_days']
    

    full_set['year'] = full_set.index.year
    full_set['month'] = full_set.index.month
    full_set['day'] = full_set.index.day
    full_set['day_of_week'] = full_set.index.weekday
    full_set['week_of_year'] = (full_set.index.isocalendar()['week']).astype('int')
    for i in range(1, 8):
        full_set[f'close_lag_{i}'] = full_set['close'].shift(i).bfill()
        full_set[f'volume_lag_{i}'] = full_set['volume'].shift(i).bfill()
        full_set[f'diff_close_value_lag_{i}'] = full_set['diff_close_value'].shift(i).bfill()
    full_set['growth_lag_7'] = full_set['growth'].shift(7).bfill()
    
    
    for window in [3, 5, 7]:
        close_agg = pd.DataFrame(round(full_set['close'].rolling(window=window, closed='left').agg(
            ('max', 'min', 'mean')
        )))
        close_agg.columns = [f'close_max_{window}_days', f'close_min_{window}_days', f'close_mean_{window}_days']
        
        full_set = full_set = pd.concat([full_set, close_agg], axis=1)
#         full_set = full_set.loc[:test_end.strftime("%Y%m%d"),]
      
        
    path_to_curr = f"/diploma_info/datalake/currency_rates.csv"
    currencies = pd.read_csv(
        path_to_curr,
        parse_dates=["date"],
        date_parser=date_parse,
        index_col=["date"],
    )
    
    currencies = pd.concat([currencies.dropna(), pd.DataFrame(None, index=new_dates)])
    currencies = currencies[~currencies.index.duplicated(keep='first')]
    currencies.index.name = 'date'
    
    for col in currencies.columns:
        currencies[f'{col}_lag_1'] = currencies[col].shift(1)
        currencies = currencies.drop([col], axis=1)
        
    full_set = pd.merge(full_set, currencies, on='date')    
    
    full_set = full_set.fillna(0)

    
    path_to_inf = f"/diploma_info/datalake/usa_inflation_rate.csv"
    inflation = pd.read_csv(
        path_to_inf,
        parse_dates=["date"],
        date_parser=date_parse,
        index_col=["date"],
    )
    for idx in inflation.index:
        full_set.loc[idx, 'usa_inflation_%'] = inflation.loc[idx, 'value']
    full_set = full_set.sort_index()

    full_set['usa_inflation_%'] = full_set['usa_inflation_%'].ffill()
    full_set = full_set.dropna().sort_index()   
        
    full_set.to_csv(f"/diploma_info/datalake/processed_data/{company}_{period}.csv")

In [6]:
full_set

,open,high,low,close,adj_close,volume,diff_close_value,diff_open_value,open-prev_close,(close-open)_lag_1,...,close_max_7_days,close_min_7_days,close_mean_7_days,1_USD_to_EUR_lag_1,1_EUR_to_USD_lag_1,1_UAH_to_USD_lag_1,1_USD_to_UAH_lag_1,1_EUR_to_UAH_lag_1,1_UAH_to_EUR_lag_1,usa_inflation_%
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,49.151428,50.331429,48.731430,49.848572,49.848572,13475000.0,1.047142,0.118572,0.349998,-0.231426,...,49.0,48.0,49.0,0.8265,1.2098,0.06322,15.819,19.138,0.05225,0.76
2015-01-05,49.258572,49.258572,47.147144,47.311428,47.311428,18165000.0,-2.537144,0.107143,-0.590000,0.697144,...,50.0,48.0,49.0,0.8370,1.1946,0.06265,15.962,18.900,0.05243,0.76
2015-01-06,47.347141,47.639999,45.661430,46.501431,46.501431,16037700.0,-0.809998,-1.911430,0.035713,-1.947144,...,50.0,47.0,49.0,0.8391,1.1918,0.06322,15.818,18.852,0.05305,0.76
2015-01-07,47.347141,47.421429,46.271427,46.742859,46.742859,9849700.0,0.241428,0.000000,0.845711,-0.845711,...,50.0,47.0,48.0,0.8377,1.1937,0.06317,15.830,18.896,0.05292,0.76
2015-01-08,47.119999,47.835712,46.478573,47.779999,47.779999,9601900.0,1.037140,-0.227142,0.377140,-0.604282,...,50.0,47.0,48.0,0.8461,1.1819,0.06317,15.830,18.709,0.05345,0.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-08,601.630005,618.219971,601.630005,609.469971,609.469971,3093900.0,3.469971,5.349976,-4.369995,9.719971,...,606.0,551.0,573.0,0.9300,1.0753,0.02544,39.310,42.270,0.02366,3.50
2024-05-09,614.400024,615.719971,605.750000,612.090027,612.090027,2065400.0,2.620056,12.770020,4.930054,7.839966,...,609.0,551.0,580.0,0.9302,1.0750,0.02537,39.414,42.370,0.02360,3.50
2024-05-10,619.000000,623.979980,605.059998,610.869995,610.869995,2651200.0,-1.220032,4.599976,6.909973,-2.309998,...,612.0,552.0,589.0,0.9273,1.0784,0.02536,39.427,42.516,0.02352,3.50


In [7]:
full_set.describe()

,open,high,low,close,adj_close,volume,diff_close_value,diff_open_value,open-prev_close,(close-open)_lag_1,...,close_max_7_days,close_min_7_days,close_mean_7_days,1_USD_to_EUR_lag_1,1_EUR_to_USD_lag_1,1_UAH_to_USD_lag_1,1_USD_to_UAH_lag_1,1_EUR_to_UAH_lag_1,1_UAH_to_EUR_lag_1,usa_inflation_%
count,2357.000000,2357.000000,2357.000000,2357.000000,2357.000000,2.357000e+03,2357.000000,2357.000000,2357.000000,2357.000000,...,2357.000000,2357.000000,2357.000000,2357.000000,2357.000000,2357.000000,2357.000000,2357.000000,2357.000000,2357.000000
mean,309.509499,313.646762,304.192071,309.037467,309.037467,9.290132e+06,0.238468,0.240194,0.190330,0.048039,...,318.808231,297.832414,308.346627,0.893583,1.121591,0.036366,28.264968,31.596944,0.032409,2.865316
std,163.721844,165.741612,161.539012,163.628660,163.628660,8.131366e+06,9.138606,8.901557,5.751127,6.899583,...,167.897979,158.914878,163.212949,0.042314,0.052806,0.005974,4.842043,4.852376,0.005101,2.363835
min,46.021427,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-122.419983,-117.320007,-107.820007,-42.479980,...,0.000000,0.000000,0.000000,0.799300,0.959600,0.025130,15.735000,17.627000,0.023370,-0.200000
25%,150.600006,152.559998,148.000000,150.179993,150.179993,4.627700e+06,-3.130005,-3.090004,-1.040009,-2.829987,...,153.000000,146.000000,149.000000,0.861600,1.086500,0.035170,25.788000,28.698000,0.029880,1.370000
50%,318.970001,323.880005,312.799988,317.940002,317.940002,6.949000e+06,0.080002,0.130005,0.160004,-0.059998,...,330.000000,306.000000,318.000000,0.894000,1.118600,0.037040,26.995000,31.028000,0.032230,2.180000
75%,430.850006,437.290009,425.540009,429.700012,429.700012,1.121500e+07,3.630005,3.820007,1.639984,3.070007,...,441.000000,414.000000,428.000000,0.920400,1.160600,0.038780,28.437000,33.466000,0.034850,3.500000
max,692.349976,700.989990,686.090027,691.690002,691.690002,1.333875e+08,84.570038,64.419983,63.649994,55.030029,...,692.000000,663.000000,680.000000,1.042100,1.251100,0.063550,39.795000,42.794000,0.056590,9.060000


In [8]:
full_set[full_set['open-prev_close'] == full_set['open-prev_close'].min()]

,open,high,low,close,adj_close,volume,diff_close_value,diff_open_value,open-prev_close,(close-open)_lag_1,...,close_max_7_days,close_min_7_days,close_mean_7_days,1_USD_to_EUR_lag_1,1_EUR_to_USD_lag_1,1_UAH_to_USD_lag_1,1_USD_to_UAH_lag_1,1_EUR_to_UAH_lag_1,1_UAH_to_EUR_lag_1,usa_inflation_%
date,,,,,,,,,,,,,,,,,,,,,
2022-01-21,400.429993,409.149994,379.98999,397.5,397.5,58904300.0,-110.75,-117.320007,-107.820007,-9.5,...,541.0,508.0,523.0,0.8839,1.1313,0.0353,28.327,32.047,0.0312,7.04


In [9]:
full_set[~((full_set['diff_close_value'] < -15) | (full_set['diff_close_value'] > 15))] \
.mean()[['diff_close_value', 'open-prev_close', '(close-open)_lag_1']]

diff_close_value      0.176958
open-prev_close       0.202410
(close-open)_lag_1    0.086830
dtype: float64

In [10]:
full_set[~((full_set['open-prev_close'] < -10) | (full_set['open-prev_close'] > 10))] \
.mean()[['diff_close_value', 'open-prev_close', '(close-open)_lag_1']]

diff_close_value      0.246185
open-prev_close       0.219205
(close-open)_lag_1    0.088788
dtype: float64

In [11]:
full_set[~((full_set['(close-open)_lag_1'] < -15) | (full_set['(close-open)_lag_1'] > 15))] \
.mean()[['diff_close_value', 'open-prev_close', '(close-open)_lag_1']]

diff_close_value      0.329135
open-prev_close       0.236298
(close-open)_lag_1    0.041309
dtype: float64

In [12]:
full_set.new_open

date
2015-01-02     49.095712
2015-01-05     49.545237
2015-01-06     49.440000
2015-01-07     47.909046
2015-01-08     47.948570
                 ...    
2024-05-08    609.700033
2024-05-09    613.506673
2024-05-10    625.260030
2024-05-13    626.573324
2024-05-14    613.949992
Name: new_open, Length: 2357, dtype: float64